[Dados FINBRA](https://www.gov.br/tesouronacional/pt-br/estados-e-municipios/dados-consolidados/finbra-financas-municipais)
- Despesas Orçamentarias
- Despesas por Funcao

[Codigos dos Múnicipios](https://www.ibge.gov.br/explica/codigos-dos-municipios.php)

[Codigo dos múnicipios utilizados](https://github.com/betafcc/Municipios-Brasileiros-TSE)

[RAIS](https://pdet.mte.gov.br/microdados-rais-e-caged)

[IBGE](https://www.ibge.gov.br/estatisticas/sociais/saude/10586-pesquisa-de-informacoes-basicas-municipais.html?=&t=downloads)

[Drive](https://drive.google.com/drive/folders/1y3TIEHYXW3C6iG_HWsyPsSVPnBkis3hL?usp=sharing)



# Imports

In [1]:
import os
import pandas as pd

In [2]:
import re
from unidecode import unidecode

# Load

In [3]:
pd.set_option('display.max_columns',9999)

In [4]:
proj_root = '\\'.join(os.getcwd().split('\\')[:-1])

In [5]:
rootdir = proj_root+'/Dados/raw'

finbra_despesas_orc = pd.DataFrame()
finbra_despesas_porfunc = pd.DataFrame()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if 'finbra' in file:
            ano = file.split('_')[-1].replace('.csv','')
            full_path = os.path.join(subdir, file)
            temp = pd.read_csv(full_path,encoding='latin',sep=';',header=3,decimal=',')
            temp['ano'] = ano
            if 'DespesasOrca' in file:
                if finbra_despesas_orc.shape[0]==0:
                    finbra_despesas_orc = temp.copy()
                else:
                    finbra_despesas_orc = pd.concat([finbra_despesas_orc,temp],ignore_index=True)
            elif 'DespesasporFunc' in file:
                if finbra_despesas_porfunc.shape[0]==0:
                    finbra_despesas_porfunc = temp.copy()
                else:
                    finbra_despesas_porfunc = pd.concat([finbra_despesas_porfunc,temp],ignore_index=True)
                
            else:
                continue

# Utils

In [5]:
def treat_text(x):
    x = unidecode(x.lower())
    x = re.sub("[\'._-]",'',x)
    x = x.replace("'",'')
    x = re.sub('^[ ]+','',x)
    x = re.sub('^[0-9]+','',x)
    x = re.sub('^[ ]+','',x)
    x = re.sub('[ ]+',' ',x)
    return x

In [6]:
def find_section(sections,x):
    for s in sections:
        s_n = s.split(' ')[0]
        if x[:len(s_n)]==s_n:
            return s
    return x

def return_sections(serie):
    
    sections = serie.loc[serie.apply(lambda x: len(re.findall(('(^[0-9]{2}[^.])|(^FU[0-9]{2}[ ])'),x))>0)].unique()
    return sections

def get_section_serie(serie):
    sections = return_sections(serie)
    return serie.apply(lambda x: find_section(sections,x))

In [7]:
def get_names(df,colunas,level):
    ret_col=[]
    for d in df.columns:
        if(d[level] in colunas):
            ret_col.append(d)
    return ret_col

In [8]:
def treat_idx(x):
    x = re.sub('(^[ ]+)|([ ]$)','',x.split('-')[0])
    if len(x)==12:
        return x+'.00'
    else:
        return x
    
def split_colunas(serie):
    
    idx = serie.apply(lambda x: treat_idx(x))
    nome = serie.apply(lambda x: re.sub('(^[ ]+)|([ ]$)','','-'.join(x.split('-')[1:])).lower())
    return idx,nome

In [9]:
def treat_id(idx,level):
    try:        
        return int(''.join(idx.split('.')[level:]))==0
    except:
        return True
def compare(x,sections=['']):
    for s in sections:
        if x[:3]==s[:3]:
            return s
    return x


def return_serie_sections_orc(df,idx='id_Conta',nome='nome_Conta',sections=['']):
    return df.apply(
        lambda x: compare(x[idx],sections)
        ,axis=1
    )
    
    
def return_sections_orc(df,idx='id_Conta',nome='nome_Conta',level=2):
    ind_idx = df.loc[df[idx].apply(lambda x: treat_id(x,level))].index
    
    return df.iloc[ind_idx][idx].unique()


def get_sections_orc(df,idx='id_Conta',nome='nome_Conta',level=2):
    sections = return_sections_orc(df,idx,nome,level)
    return  return_serie_sections_orc(df,idx,nome,sections)
    

# Prep + Exploração

## FINBRA DESPESAS POR FUNÇÃO

In [11]:
finbra_despesas_porfunc['Conta'] = \
finbra_despesas_porfunc['Conta'].replace('Despesas (Exceto Intraorçamentárias)','Despesas Exceto Intraorçamentárias')

In [12]:
return_sections(finbra_despesas_porfunc['Conta'])

array(['01 - Legislativa', '02 - Judiciária', '04 - Administração',
       '06 - Segurança Pública', '08 - Assistência Social', '10 - Saúde',
       '12 - Educação', '13 - Cultura', '14 - Direitos da Cidadania',
       '15 - Urbanismo', '17 - Saneamento', '18 - Gestão Ambiental',
       '20 - Agricultura', '23 - Comércio e Serviços',
       '24 - Comunicações', '25 - Energia', '26 - Transporte',
       '27 - Desporto e Lazer', '28 - Encargos Especiais',
       '16 - Habitação', '22 - Indústria', '19 - Ciência e Tecnologia',
       '03 - Essencial à Justiça', '09 - Previdência Social',
       '11 - Trabalho', '05 - Defesa Nacional',
       '07 - Relações Exteriores', '21 - Organização Agrária',
       'FU12 - Demais Subfunções', 'FU17 - Demais Subfunções',
       'FU15 - Demais Subfunções', 'FU20 - Demais Subfunções',
       'FU26 - Demais Subfunções', 'FU02 - Demais Subfunções',
       'FU04 - Demais Subfunções', 'FU14 - Demais Subfunções',
       'FU16 - Demais Subfunções', 'FU06 - De

In [13]:
finbra_despesas_porfunc['Seção'] = get_section_serie(finbra_despesas_porfunc['Conta'])

In [14]:
#finbra_despesas_porfunc['Conta_tratada'] = finbra_despesas_porfunc['Conta'].map(treat_text)

#finbra_despesas_porfunc['Coluna_tratada'] = finbra_despesas_porfunc['Coluna'].map(treat_text)

In [15]:
finbra_despesas_porfunc.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,ano,Seção
0,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,Despesas Exceto Intraorçamentárias,siconfi-cor_TotalDespesas,15743045.91,2016,Despesas Exceto Intraorçamentárias
1,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,01 - Legislativa,siconfi-cor_TotalDespesas,442406.08,2016,01 - Legislativa
2,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,01.031 - Ação Legislativa,siconfi-cor_TotalDespesas,442406.08,2016,01 - Legislativa
3,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,02 - Judiciária,siconfi-cor_TotalDespesas,496720.66,2016,02 - Judiciária
4,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,02.061 - Ação Judiciária,siconfi-cor_TotalDespesas,496720.66,2016,02 - Judiciária


In [16]:
finbra_despesas_porfunc.groupby(
    ['ano','Cod.IBGE','Identificador da Conta','Coluna','Conta']
)['Valor'].count().sort_values().tail()

ano   Cod.IBGE  Identificador da Conta     Coluna               Conta            
2017  1503093   siconfi-cor_TotalDespesas  Despesas Empenhadas  Demais Subfunções    5
                                           Despesas Liquidadas  Demais Subfunções    5
      4319505   siconfi-cor_TotalDespesas  Despesas Empenhadas  Demais Subfunções    7
                                           Despesas Liquidadas  Demais Subfunções    7
                                           Despesas Pagas       Demais Subfunções    7
Name: Valor, dtype: int64

In [17]:
finbra_despesas_porfunc.loc[
    (finbra_despesas_porfunc['ano']=='2016') & \
    (finbra_despesas_porfunc['Cod.IBGE']==4319505) & \
    (finbra_despesas_porfunc['Coluna'].str.contains('Despesas Pagas')) & \
    (finbra_despesas_porfunc['Conta'].str.contains('Demais Subfunções'))
]

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,ano,Seção
3542,Prefeitura Municipal de São Sebastião do Caí - RS,4319505,RS,24676,Despesas Pagas,10.999 - Demais Subfunções Saúde,siconfi-cor_TotalDespesas,324950.27,2016,10 - Saúde
3553,Prefeitura Municipal de São Sebastião do Caí - RS,4319505,RS,24676,Despesas Pagas,13.999 - Demais Subfunções Cultura,siconfi-cor_TotalDespesas,97.58,2016,13 - Cultura
3555,Prefeitura Municipal de São Sebastião do Caí - RS,4319505,RS,24676,Despesas Pagas,14.999 - Demais Subfunções Direitos da Cidadania,siconfi-cor_TotalDespesas,1935.70,2016,14 - Direitos da Cidadania
3562,Prefeitura Municipal de São Sebastião do Caí - RS,4319505,RS,24676,Despesas Pagas,17.999 - Demais Subfunções Saneamento,siconfi-cor_TotalDespesas,1025973.00,2016,17 - Saneamento
3570,Prefeitura Municipal de São Sebastião do Caí - RS,4319505,RS,24676,Despesas Pagas,23.999 - Demais Subfunções Comércio e Serviços,siconfi-cor_TotalDespesas,37273.78,2016,23 - Comércio e Serviços


### Sanity check:
### Despesas exceto intraorçamentarias é o total das despesas
##### Total da despesa Geral = Despesas de capital + Despesas correntes = Soma todas despesas

In [18]:
amostra = finbra_despesas_porfunc.loc[finbra_despesas_porfunc['Conta']==finbra_despesas_porfunc['Seção']]
amostra = amostra.loc[amostra['Cod.IBGE']==1100015]
amostra = amostra.loc[amostra['Coluna']=='Despesas Pagas']
amostra = amostra.loc[amostra['Seção'].str.slice(0,2)!='FU']
amostra.reset_index(drop=True,inplace=True)

In [19]:
amostra

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,ano,Seção
0,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,25578,Despesas Pagas,Despesas Exceto Intraorçamentárias,siconfi-cor_TotalDespesas,50218466.43,2016,Despesas Exceto Intraorçamentárias
1,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,25578,Despesas Pagas,01 - Legislativa,siconfi-cor_TotalDespesas,2204087.60,2016,01 - Legislativa
2,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,25578,Despesas Pagas,04 - Administração,siconfi-cor_TotalDespesas,9814764.71,2016,04 - Administração
3,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,25578,Despesas Pagas,08 - Assistência Social,siconfi-cor_TotalDespesas,451656.32,2016,08 - Assistência Social
4,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,25578,Despesas Pagas,10 - Saúde,siconfi-cor_TotalDespesas,13984336.50,2016,10 - Saúde
...,...,...,...,...,...,...,...,...,...,...
56,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,22945,Despesas Pagas,20 - Agricultura,siconfi-cor_TotalDespesas,176784.80,2020,20 - Agricultura
57,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,22945,Despesas Pagas,23 - Comércio e Serviços,siconfi-cor_TotalDespesas,3785.40,2020,23 - Comércio e Serviços
58,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,22945,Despesas Pagas,26 - Transporte,siconfi-cor_TotalDespesas,1427136.18,2020,26 - Transporte
59,Prefeitura Municipal de Alta Floresta D'Oeste ...,1100015,RO,22945,Despesas Pagas,27 - Desporto e Lazer,siconfi-cor_TotalDespesas,76784.14,2020,27 - Desporto e Lazer


In [20]:
amostra.loc[amostra['Conta']=='Despesas Exceto Intraorçamentárias'][['ano','Valor']]

,ano,Valor
0,2016,50218466.43
12,2017,55089113.55
24,2018,53048050.35
36,2019,57863292.60
48,2020,58410000.87


In [21]:
amostra.loc[amostra['Conta']!='Despesas Exceto Intraorçamentárias'].groupby('ano')['Valor'].sum().astype(int)

ano
2016    50218466
2017    55089113
2018    53048050
2019    57863292
2020    58410000
Name: Valor, dtype: int32

# Pivoteamento

In [22]:
finbra_despesas_unidas = finbra_despesas_porfunc.groupby(
    ['ano','Cod.IBGE','Identificador da Conta','Seção','Coluna','Conta']
)['Valor'].sum().reset_index()

finbra_despesas_por_func_pivot = finbra_despesas_unidas.pivot(
    index=['ano','Cod.IBGE']
    ,columns=['Coluna','Seção','Conta']
    ,values='Valor'
)

finbra_despesas_por_func_pivot = finbra_despesas_por_func_pivot[sorted(finbra_despesas_por_func_pivot.columns)]

finbra_despesas_por_func_pivot.fillna(0,inplace=True)

finbra_despesas_por_func_pivot.head()

Seções = [
    '08 - Assistência Social'
    ,'10 - Saúde'
    ,'12 - Educação'
    ,'13 - Cultura'
    ,'28 - Transferências para a Educação Básica'
    ,'Despesas Exceto Intraorçamentárias'
]

finbra_despesas_por_func_pivot = finbra_despesas_por_func_pivot[
    get_names(finbra_despesas_por_func_pivot,['Despesas Empenhadas','Despesas Pagas'],level=0)
]
finbra_despesas_por_func_pivot = finbra_despesas_por_func_pivot[
    get_names(finbra_despesas_por_func_pivot,Seções,level=1)
]

In [23]:
finbra_despesas_unidas_soma = finbra_despesas_porfunc.groupby(
    ['Cod.IBGE','Identificador da Conta','Seção','Coluna','Conta']
)['Valor'].sum().reset_index()

finbra_despesas_por_func_pivot_soma = finbra_despesas_unidas_soma.pivot(
    index=['Cod.IBGE']
    ,columns=['Coluna','Seção','Conta']
    ,values='Valor'
)

finbra_despesas_por_func_pivot_soma = finbra_despesas_por_func_pivot_soma[sorted(finbra_despesas_por_func_pivot_soma.columns)]

finbra_despesas_por_func_pivot_soma.fillna(0,inplace=True)

finbra_despesas_por_func_pivot_soma.head()

Seções = [
    '08 - Assistência Social'
    ,'10 - Saúde'
    ,'12 - Educação'
    ,'13 - Cultura'
    ,'28 - Transferências para a Educação Básica'
    ,'Despesas Exceto Intraorçamentárias'
]

finbra_despesas_por_func_pivot_soma = finbra_despesas_por_func_pivot_soma[
    get_names(finbra_despesas_por_func_pivot_soma,['Despesas Empenhadas','Despesas Pagas'],level=0)
]
finbra_despesas_por_func_pivot_soma = finbra_despesas_por_func_pivot_soma[
    get_names(finbra_despesas_por_func_pivot_soma,Seções,level=1)
]

In [24]:
with pd.ExcelWriter(proj_root + '/Dados/finbra_despesas_por_func_tratada.xlsx') as exw:
    finbra_despesas_por_func_pivot.to_excel(
        exw
        ,sheet_name='Visao ano a ano'
        ,engine='openpyxl'
    )

    finbra_despesas_por_func_pivot_soma.to_excel(
        exw
        ,sheet_name='Visao resumida'
        ,engine='openpyxl'
    )

## FINBRA DESPESAS ORÇAMENTÁRIAS

In [25]:
finbra_despesas_orc['Conta'].replace(
    '4.4.60.00.00 - Transferências a Instituições Privadas com Fins Lucrativos',
    '4.4.60.00.00.01 - Transferências a Instituições Privadas com Fins Lucrativos',
    inplace=True
)

finbra_despesas_orc['Conta'].replace(
    '4.4.60.00.00.00 - Transferências a Instituições Privadas com Fins Lucrativos',
    '4.4.60.00.00.01 - Transferências a Instituições Privadas com Fins Lucrativos',
    inplace=True
)

finbra_despesas_orc['id_Conta'],finbra_despesas_orc['nome_Conta'] = split_colunas(finbra_despesas_orc['Conta'])

finbra_despesas_orc['nome_Conta'] = finbra_despesas_orc['nome_Conta'].replace('','Total Geral da Despesa')

finbra_despesas_orc['id_Seção'] = get_sections_orc(finbra_despesas_orc)

de_para_id_nome = finbra_despesas_orc[['id_Conta','nome_Conta']].drop_duplicates()
de_para_id_nome = de_para_id_nome.set_index('id_Conta')['nome_Conta'].to_dict()

finbra_despesas_orc['nome_Seção'] = finbra_despesas_orc['id_Seção'].map(de_para_id_nome)

In [26]:
finbra_despesas_orc.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,ano,id_Conta,nome_Conta,id_Seção,nome_Seção
0,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,Total Geral da Despesa,siconfi-cor_TotalDespesas,15743045.91,2016,Total Geral da Despesa,Total Geral da Despesa,Total Geral da Despesa,Total Geral da Despesa
1,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,3.0.00.00.00.00 - Despesas Correntes,siconfi-cor_DO3.0.00.00.00.00,14841925.71,2016,3.0.00.00.00.00,despesas correntes,3.0.00.00.00.00,despesas correntes
2,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,3.1.00.00.00.00 - Pessoal e Encargos Sociais,siconfi-cor_DO3.1.00.00.00.00,8541410.12,2016,3.1.00.00.00.00,pessoal e encargos sociais,3.1.00.00.00.00,pessoal e encargos sociais
3,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,3.1.90.00.00.00 - Aplicações Diretas,siconfi-cor_DO3.1.90.00.00.00,8541410.12,2016,3.1.90.00.00.00,aplicações diretas,3.1.00.00.00.00,pessoal e encargos sociais
4,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4061,Despesas Empenhadas,3.1.90.11.00.00 - Vencimentos e Vantagens Fixa...,siconfi-cor_DO3.1.90.11.00.00,7117169.61,2016,3.1.90.11.00.00,vencimentos e vantagens fixas - pessoal civil,3.1.00.00.00.00,pessoal e encargos sociais


### Sanity check:
### Total de despesas é dividido da seguinte maneira
##### Total da despesa Geral = Despesas de capital + Despesas correntes = Soma todas despesas

In [27]:
amostra = finbra_despesas_orc.loc[finbra_despesas_orc['nome_Conta']==finbra_despesas_orc['nome_Seção']]
amostra = amostra.loc[amostra['Cod.IBGE']==4305405]
amostra = amostra.loc[amostra['Coluna']=='Despesas Pagas']
amostra = amostra.loc[amostra['ano']=='2017']
amostra.reset_index(drop=True,inplace=True)

In [28]:
amostra.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,ano,id_Conta,nome_Conta,id_Seção,nome_Seção
0,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4043,Despesas Pagas,Total Geral da Despesa,siconfi-cor_TotalDespesas,16787793.23,2017,Total Geral da Despesa,Total Geral da Despesa,Total Geral da Despesa,Total Geral da Despesa
1,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4043,Despesas Pagas,3.0.00.00.00.00 - Despesas Correntes,siconfi-cor_DO3.0.00.00.00.00,15990783.24,2017,3.0.00.00.00.00,despesas correntes,3.0.00.00.00.00,despesas correntes
2,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4043,Despesas Pagas,3.1.00.00.00.00 - Pessoal e Encargos Sociais,siconfi-cor_DO3.1.00.00.00.00,8809741.63,2017,3.1.00.00.00.00,pessoal e encargos sociais,3.1.00.00.00.00,pessoal e encargos sociais
3,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4043,Despesas Pagas,3.2.00.00.00.00 - Juros e Encargos da Dívida,siconfi-cor_DO3.2.00.00.00.00,2452.27,2017,3.2.00.00.00.00,juros e encargos da dívida,3.2.00.00.00.00,juros e encargos da dívida
4,Prefeitura Municipal de Chiapetta - RS,4305405,RS,4043,Despesas Pagas,3.3.00.00.00.00 - Outras Despesas Correntes,siconfi-cor_DO3.3.00.00.00.00,7178589.34,2017,3.3.00.00.00.00,outras despesas correntes,3.3.00.00.00.00,outras despesas correntes


In [29]:
amostra.loc[amostra['nome_Conta']=='Total Geral da Despesa']['Valor'].iloc[0]

16787793.23

In [30]:
amostra.loc[amostra['nome_Conta']=='despesas correntes']['Valor'].iloc[0] + \
amostra.loc[amostra['nome_Conta']=='despesas de capital']['Valor'].iloc[0]

16787793.23

In [31]:
amostra.loc[
    (amostra['nome_Conta']!='despesas correntes') & \
    (amostra['nome_Conta']!='despesas de capital') & \
    (amostra['nome_Conta']!='Total Geral da Despesa')
]['Valor'].sum()

16787793.23

# Geração do Pivot

In [32]:
finbra_despesas_orc = finbra_despesas_orc.groupby(
    ['ano','Cod.IBGE','nome_Seção','Coluna','nome_Conta']
)['Valor'].sum().reset_index()

finbra_despesas_orc_pivot = finbra_despesas_orc.pivot(
    index=['ano','Cod.IBGE']
    ,columns=['Coluna','nome_Seção','nome_Conta']
    ,values='Valor'
)

finbra_despesas_orc_pivot = finbra_despesas_orc_pivot[sorted(finbra_despesas_orc_pivot.columns)]

finbra_despesas_orc_pivot.fillna(0,inplace=True)

finbra_despesas_orc_pivot = finbra_despesas_orc_pivot[
    get_names(finbra_despesas_orc_pivot,['Despesas Empenhadas','Despesas Pagas'],level=0)
]

print(finbra_despesas_orc_pivot.columns.get_level_values(1).unique())

for c in finbra_despesas_orc_pivot:
    if 'sem fins luc' in c[2].lower():
        print(c)

Seções = [
    'Total Geral da Despesa'
    ,'despesas correntes'
    ,'pessoal e encargos sociais'
    ,'despesas de capital'
]

Contas = [
    'transferências a instituições privadas sem fins lucrativos'
]

finbra_despesas_orc_pivot_1 = finbra_despesas_orc_pivot[
    get_names(finbra_despesas_orc_pivot,Seções,level=1)
]

finbra_despesas_orc_pivot_2 = finbra_despesas_orc_pivot[
    get_names(finbra_despesas_orc_pivot,Contas,level=2)
]

cols = finbra_despesas_orc_pivot_2.columns
for c in cols:
    if c in finbra_despesas_orc_pivot_1.columns:
        finbra_despesas_orc_pivot_2.drop(c,axis=1,inplace=True)

finbra_despesas_orc_pivot_3 = finbra_despesas_orc_pivot_1.merge(
    finbra_despesas_orc_pivot_2
    ,how='inner'
    ,left_index=True
    ,right_index=True
)

finbra_despesas_orc_pivot_3 = finbra_despesas_orc_pivot_3[sorted(finbra_despesas_orc_pivot_3.columns)]

Index(['Total Geral da Despesa', 'amortização da dívida', 'despesas correntes',
       'despesas de capital', 'inversões financeiras', 'investimentos',
       'juros e encargos da dívida', 'outras despesas correntes',
       'pessoal e encargos sociais'],
      dtype='object', name='nome_Seção')
('Despesas Empenhadas', 'inversões financeiras', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'investimentos', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'juros e encargos da dívida', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'outras despesas correntes', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'pessoal e encargos sociais', 'transferências a instituições sem fins lucrativos')
('Despesas Pagas', 'inversões financeiras', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Pagas', 'investimentos', 'transfer

In [33]:
finbra_despesas_orc_soma = finbra_despesas_orc.groupby(
    ['Cod.IBGE','nome_Seção','Coluna','nome_Conta']
)['Valor'].sum().reset_index()

finbra_despesas_orc_pivot_soma = finbra_despesas_orc_soma.pivot(
    index=['Cod.IBGE']
    ,columns=['Coluna','nome_Seção','nome_Conta']
    ,values='Valor'
)

finbra_despesas_orc_pivot_soma = finbra_despesas_orc_pivot_soma[sorted(finbra_despesas_orc_pivot_soma.columns)]

finbra_despesas_orc_pivot_soma.fillna(0,inplace=True)

finbra_despesas_orc_pivot_soma = finbra_despesas_orc_pivot_soma[
    get_names(finbra_despesas_orc_pivot_soma,['Despesas Empenhadas','Despesas Pagas'],level=0)
]

print(finbra_despesas_orc_pivot_soma.columns.get_level_values(1).unique())

for c in finbra_despesas_orc_pivot_soma:
    if 'sem fins luc' in c[2].lower():
        print(c)

Seções = [
    'Total Geral da Despesa'
    ,'despesas correntes'
    ,'pessoal e encargos sociais'
    ,'despesas de capital'
]

Contas = [
    'transferências a instituições privadas sem fins lucrativos'
]

finbra_despesas_orc_pivot_soma_1 = finbra_despesas_orc_pivot_soma[
    get_names(finbra_despesas_orc_pivot_soma,Seções,level=1)
]

finbra_despesas_orc_pivot_soma_2 = finbra_despesas_orc_pivot_soma[
    get_names(finbra_despesas_orc_pivot_soma,Contas,level=2)
]

cols = finbra_despesas_orc_pivot_soma_2.columns
for c in cols:
    if c in finbra_despesas_orc_pivot_soma_1.columns:
        finbra_despesas_orc_pivot_soma_2.drop(c,axis=1,inplace=True)

finbra_despesas_orc_pivot_soma_3 = finbra_despesas_orc_pivot_soma_1.merge(
    finbra_despesas_orc_pivot_soma_2
    ,how='inner'
    ,left_index=True
    ,right_index=True
)

finbra_despesas_orc_pivot_soma_3 = finbra_despesas_orc_pivot_soma_3[sorted(finbra_despesas_orc_pivot_soma_3.columns)]

Index(['Total Geral da Despesa', 'amortização da dívida', 'despesas correntes',
       'despesas de capital', 'inversões financeiras', 'investimentos',
       'juros e encargos da dívida', 'outras despesas correntes',
       'pessoal e encargos sociais'],
      dtype='object', name='nome_Seção')
('Despesas Empenhadas', 'inversões financeiras', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'investimentos', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'juros e encargos da dívida', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'outras despesas correntes', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Empenhadas', 'pessoal e encargos sociais', 'transferências a instituições sem fins lucrativos')
('Despesas Pagas', 'inversões financeiras', 'transferências a instituições privadas sem fins lucrativos')
('Despesas Pagas', 'investimentos', 'transfer

In [34]:
finbra_despesas_orc_pivot_3.head()

Coluna           Despesas Empenhadas                                         \
nome_Seção    Total Geral da Despesa despesas correntes despesas de capital   
nome_Conta    Total Geral da Despesa despesas correntes despesas de capital   
ano  Cod.IBGE                                                                 
2016 1100015            5.430962e+07       4.693673e+07          7372890.14   
     1100023            2.191308e+08       1.994629e+08         19667816.83   
     1100031            1.954305e+07       1.834665e+07          1196398.80   
     1100049            1.699705e+08       1.518830e+08         18087489.43   
     1100056            4.557934e+07       3.526282e+07         10316526.17   

Coluna                                                                    \
nome_Seção                                         inversões financeiras   
nome_Conta    transferências a instituições privadas sem fins lucrativos   
ano  Cod.IBGE                                                              
2016 1100015                                                 0.0           
     1100023                                                 0.0           
     1100031                                                 0.0           
     1100049                                                 0.0           
     1100056                                                 0.0           

Coluna                                                                    \
nome_Seção                                                 investimentos   
nome_Conta    transferências a instituições privadas sem fins lucrativos   
ano  Cod.IBGE                                                              
2016 1100015                                                 0.0           
     1100023                                             84941.0           
     1100031                                                 0.0           
     1100049                                                 0.0           
     1100056                                                 0.0           

Coluna                                                                    \
nome_Seção                                    juros e encargos da dívida   
nome_Conta    transferências a instituições privadas sem fins lucrativos   
ano  Cod.IBGE                                                              
2016 1100015                                                 0.0           
     1100023                                                 0.0           
     1100031                                                 0.0           
     1100049                                                 0.0           
     1100056                                                 0.0           

Coluna                                                                    \
nome_Seção                                     outras despesas correntes   
nome_Conta    transferências a instituições privadas sem fins lucrativos   
ano  Cod.IBGE                                                              
2016 1100015                                           474506.57           
     1100023                                         13507982.74           
     1100031                                                0.00           
     1100049                                           355606.47           
     1100056                                            73851.79           

Coluna                                              \
nome_Seção    pessoal e encargos sociais             
nome_Conta                 a classificar a definir   
ano  Cod.IBGE                                        
2016 1100015                        0.00       0.0   
     1100023                  8001614.26       0.0   
     1100031                   413350.00       0.0   
     1100049                        0.00       0.0   
     1100056                        0.00       0.0   

Coluna                                                       

In [35]:
finbra_despesas_orc_pivot_soma_3.head()

Coluna        Despesas Empenhadas                                         \
nome_Seção Total Geral da Despesa despesas correntes despesas de capital   
nome_Conta Total Geral da Despesa despesas correntes despesas de capital   
Cod.IBGE                                                                   
1100015              2.983507e+08       2.596092e+08        3.874144e+07   
1100023              1.198012e+09       1.072716e+09        1.252967e+08   
1100031              1.139387e+08       1.002138e+08        1.372489e+07   
1100049              1.015979e+09       8.716085e+08        1.443705e+08   
1100056              2.421425e+08       2.108330e+08        3.130948e+07   

Coluna                                                                 \
nome_Seção                                      inversões financeiras   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                   0.0           
1100023                                                   0.0           
1100031                                                   0.0           
1100049                                                   0.0           
1100056                                                   0.0           

Coluna                                                                 \
nome_Seção                                              investimentos   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                  0.00           
1100023                                           12554396.21           
1100031                                                  0.00           
1100049                                                  0.00           
1100056                                                  0.00           

Coluna                                                                 \
nome_Seção                                 juros e encargos da dívida   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                   0.0           
1100023                                                   0.0           
1100031                                                   0.0           
1100049                                                   0.0           
1100056                                                   0.0           

Coluna                                                                 \
nome_Seção                                  outras despesas correntes   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                            1463874.57           
1100023                                           51690700.29           
1100031                                                  0.00           
1100049                                            5278134.86           
1100056                                             898820.27           

Coluna                                           \
nome_Seção pessoal e encargos sociais             
nome_Conta              a classificar a definir   
Cod.IBGE                                          
1100015                          0.00       0.0   
1100023                   14284655.72       0.0   
1100031                    1339700.00       0.0   
1100049                          0.00       0.0   
1100056                          0.00       0.0   

Coluna                                                                                             \
nome_Seção                                                                                          
nome_Conta alíquota s

In [36]:
with pd.ExcelWriter(proj_root + '/Dados/finbra_despesas_orcamentarias_tratada.xlsx') as exw:
    finbra_despesas_orc_pivot_3.to_excel(
        exw
        ,sheet_name='Visao ano a ano'
        ,engine='openpyxl'
    )

    finbra_despesas_orc_pivot_soma_3.to_excel(
        exw
        ,sheet_name='Visao resumida'
        ,engine='openpyxl'
    )

# Lendo base pré processada Despesas por Função

In [72]:
finbra_despesas_por_func_pivot_soma = \
pd.read_excel(
    proj_root+'/Dados/Interim/finbra_despesas_por_func_tratada.xlsx'
    , header=[0,1,2]
    , sheet_name='Visao resumida'
    , engine='openpyxl'
    , index_col=0
)

In [73]:
finbra_despesas_por_func_pivot_soma.head()

Coluna       Despesas Empenhadas                               \
Seção    08 - Assistência Social                                
Conta    08 - Assistência Social 08.122 - Administração Geral   
Cod.IBGE                                                        
1100015               3137606.37                         0.00   
1100023              28772596.30                  13976220.53   
1100031               6156396.16                         0.00   
1100049              20538068.29                  10646988.71   
1100056               9563388.96                   5266173.79   

Coluna                                  \
Seção                                    
Conta    08.241 - Assistência ao Idoso   
Cod.IBGE                                 
1100015                            0.0   
1100023                            0.0   
1100031                       181800.0   
1100049                            0.0   
1100056                         6736.8   

Coluna                                                    \
Seção                                                      
Conta    08.242 - Assistência ao Portador de Deficiência   
Cod.IBGE                                                   
1100015                                             0.00   
1100023                                         38981.10   
1100031                                        307429.17   
1100049                                             0.00   
1100056                                             0.00   

Coluna                                                    \
Seção                                                      
Conta    08.243 - Assistência à Criança e ao Adolescente   
Cod.IBGE                                                   
1100015                                         65007.60   
1100023                                       2857226.91   
1100031                                        974134.76   
1100049                                       1342814.92   
1100056                                        894590.65   

Coluna                                     \
Seção                                       
Conta    08.244 - Assistência Comunitária   
Cod.IBGE                                    
1100015                        3072598.77   
1100023                       11900167.76   
1100031                        4693032.23   
1100049                        5952151.29   
1100056                        2215138.41   

Coluna                                                                \
Seção                                                     10 - Saúde   
Conta    08.999 - Demais Subfunções Assistência Social    10 - Saúde   
Cod.IBGE                                                               
1100015                                           0.00  8.404673e+07   
1100023                                           0.00  3.326118e+08   
1100031                                           0.00  3.371752e+07   
1100049                                        4423.40  2.375631e+08   
1100056                                     1180749.31  6.740134e+07   

Coluna                                                         \
Seção                                                           
Conta    10.122 - Administração Geral 10.301 - Atenção Básica   
Cod.IBGE                                                        
1100015                      64830.00             48470182.30   
1100023                   74881619.69             78300010.70   
1100031                          0.00              7615593.90   
1100049                   41328413.02             54412662.63   
1100056                   31594676.16             11408721.44   

Coluna                                                   \
Seção                                                     
Conta    10.302 - Assistência Hospitalar e Ambulatorial   
Cod.IBGE                                                  
1100015                                    3.478183e+07   
1100023          

# Lendo base pré processada Despesas Orçamentarias

In [69]:
finbra_despesas_orc_pivot_soma_3 = \
pd.read_excel(
    proj_root+'/Dados/Interim/finbra_despesas_orcamentarias_tratada.xlsx'
    , header=[0,1,2]
    , sheet_name='Visao resumida'
    , engine='openpyxl'
    , index_col=0
)

In [70]:
finbra_despesas_orc_pivot_soma_3.head()

Coluna        Despesas Empenhadas                                         \
nome_Seção Total Geral da Despesa despesas correntes despesas de capital   
nome_Conta Total Geral da Despesa despesas correntes despesas de capital   
Cod.IBGE                                                                   
1100015              2.983507e+08       2.596092e+08        3.874144e+07   
1100023              1.198012e+09       1.072716e+09        1.252967e+08   
1100031              1.139387e+08       1.002138e+08        1.372489e+07   
1100049              1.015979e+09       8.716085e+08        1.443705e+08   
1100056              2.421425e+08       2.108330e+08        3.130948e+07   

Coluna                                                                 \
nome_Seção                                      inversões financeiras   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                   0.0           
1100023                                                   0.0           
1100031                                                   0.0           
1100049                                                   0.0           
1100056                                                   0.0           

Coluna                                                                 \
nome_Seção                                              investimentos   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                  0.00           
1100023                                           12554396.21           
1100031                                                  0.00           
1100049                                                  0.00           
1100056                                                  0.00           

Coluna                                                                 \
nome_Seção                                 juros e encargos da dívida   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                                   0.0           
1100023                                                   0.0           
1100031                                                   0.0           
1100049                                                   0.0           
1100056                                                   0.0           

Coluna                                                                 \
nome_Seção                                  outras despesas correntes   
nome_Conta transferências a instituições privadas sem fins lucrativos   
Cod.IBGE                                                                
1100015                                            1463874.57           
1100023                                           51690700.29           
1100031                                                  0.00           
1100049                                            5278134.86           
1100056                                             898820.27           

Coluna                                           \
nome_Seção pessoal e encargos sociais             
nome_Conta              a classificar a definir   
Cod.IBGE                                          
1100015                          0.00       0.0   
1100023                   14284655.72       0.0   
1100031                    1339700.00       0.0   
1100049                          0.00       0.0   
1100056                          0.00       0.0   

Coluna                                                                                             \
nome_Seção                                                                                          
nome_Conta alíquota s

# Join com a base a ser utilizada

In [10]:
#df_base = pd.read_excel(proj_root+'/Dados/20220222 - Base de dados para rodar - valores.xlsx',engine='openpyxl')
df_base = pd.read_excel(proj_root+'/Dados/Interim/eleicoes_2020_municipios.xlsx')

In [11]:
df_base.head()

,nr_candidato_eleito,nm_candidato_eleito,nm_ue,sg_ue,nr_turno_eleito,ds_sit_tot_turno_eleito,qt_votos_nominais_validos_eleito,ds_genero_eleito,ds_grau_instrucao_eleito,ds_estado_civil_eleito,ds_cor_raca_eleito,ds_ocupacao_eleito,total_votos_municipio_eleito,%_eleito,nr_candidato_nao_eleito,nm_candidato_nao_eleito,nr_turno_nao_eleito,ds_sit_tot_turno_nao_eleito,qt_votos_nominais_validos_nao_eleito,ds_genero_nao_eleito,ds_grau_instrucao_nao_eleito,ds_estado_civil_nao_eleito,ds_cor_raca_nao_eleito,ds_ocupacao_nao_eleito,total_votos_municipio_nao_eleito,%_nao_eleito,diferenca_pp,flag_dif_5pp,flag_dif_10pp,flag_genero_diferente_eleito_nao_eleito,uf,nome_municipio,capital,codigo_ibge
0,45,BRUNO COVAS LOPES,SÃO PAULO,71072,2,ELEITO,3169121,MASCULINO,SUPERIOR COMPLETO,DIVORCIADO(A),BRANCA,PREFEITO,10675386,0.296862,50.0,GUILHERME CASTRO BOULOS,2.0,NÃO ELEITO,2168109.0,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),BRANCA,PROFESSOR DE ENSINO SUPERIOR,10675386.0,0.203094,0.093768,0,1,0,SP,SÃO PAULO,1,3550308
1,25,EDUARDO DA COSTA PAES,RIO DE JANEIRO,60011,2,ELEITO,1629319,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,OUTROS,5176341,0.314763,10.0,MARCELO BEZERRA CRIVELLA,2.0,NÃO ELEITO,913700.0,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,PREFEITO,5176341.0,0.176515,0.138248,0,0,0,RJ,RIO DE JANEIRO,1,3304557
2,55,ALEXANDRE KALIL,BELO HORIZONTE,41238,1,ELEITO,784307,MASCULINO,SUPERIOR INCOMPLETO,CASADO(A),BRANCA,EMPRESÁRIO,1237764,0.633648,28.0,BRUNO DE CASTRO ENGLER FLORENCIO DE ALMEIDA,1.0,NÃO ELEITO,123215.0,MASCULINO,SUPERIOR INCOMPLETO,SOLTEIRO(A),BRANCA,DEPUTADO,1237764.0,0.099546,0.534102,0,0,0,MG,BELO HORIZONTE,1,3106200
3,25,BRUNO SOARES REIS,SALVADOR,38490,1,ELEITO,779408,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,ADVOGADO,1214123,0.641951,13.0,DENICE SANTIAGO SANTOS DO ROSARIO,1.0,NÃO ELEITO,228942.0,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),PRETA,POLICIAL MILITAR,1214123.0,0.188566,0.453386,0,0,1,BA,SALVADOR,1,2927408
4,12,JOSÉ SARTO NOGUEIRA MOREIRA,FORTALEZA,13897,2,ELEITO,668652,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,DEPUTADO,2574469,0.259724,90.0,WAGNER SOUSA GOMES,2.0,NÃO ELEITO,624892.0,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,DEPUTADO,2574469.0,0.242727,0.016998,1,1,0,CE,FORTALEZA,1,2304400


In [13]:
de_para_cod_IBGE = pd.read_csv(
    proj_root + '/Dados/Raw/municipios_brasileiros_tse.csv'
)

In [14]:
de_para_cod_IBGE.head()

,codigo_tse,uf,nome_municipio,capital,codigo_ibge
0,1120,AC,ACRELÂNDIA,0,1200013
1,1570,AC,ASSIS BRASIL,0,1200054
2,1058,AC,BRASILÉIA,0,1200104
3,1007,AC,BUJARI,0,1200138
4,1015,AC,CAPIXABA,0,1200179


In [15]:
# df_base = df_base.merge(
#     de_para_cod_IBGE
#     ,how='left'
#     ,left_on='Cod_Munic_TSE'
#     ,right_on='codigo_tse'
#     ,suffixes=('','_depara')
# )

In [16]:
# df_base.head()

In [76]:
df_base2 = df_base.copy()

In [77]:
df_base2.columns = pd.MultiIndex.from_product([[''],[''],df_base.columns])

In [78]:
df_base2.set_index(('','','codigo_ibge'),inplace=True)

In [79]:
df_base2.index.name = 'Código IBGE'

In [80]:
df_base2

\
                                                                
            nr_candidato_eleito           nm_candidato_eleito   
Código IBGE                                                     
3550308                      45             BRUNO COVAS LOPES   
3304557                      25         EDUARDO DA COSTA PAES   
3106200                      55               ALEXANDRE KALIL   
2927408                      25             BRUNO SOARES REIS   
2304400                      12   JOSÉ SARTO NOGUEIRA MOREIRA   
...                         ...                           ...   
3507209                      14         LUIZ CARLOS RODRIGUES   
3555703                      43  KENDREA ALVES PAPILE CAVATÃO   
4112603                      11       LUCIMAR DE SOUZA MORAIS   
5101209                      17     FRANCISCO GONÇALVES NAVES   
5107883                      11         ELSON FARIAS DE SOUSA   

                                                        \
                                                         
                          nm_ue  sg_ue nr_turno_eleito   
Código IBGE                                              
3550308               SÃO PAULO  71072               2   
3304557          RIO DE JANEIRO  60011               2   
3106200          BELO HORIZONTE  41238               1   
2927408                SALVADOR  38490               1   
2304400               FORTALEZA  13897               2   
...                         ...    ...             ...   
3507209                    BORÁ  62430               1   
3555703          UNIÃO PAULISTA  72150               1   
4112603           JARDIM OLINDA  76457               1   
5101209              ARAGUAINHA  90239               1   
5107883      SERRA NOVA DOURADA  90921               1   

                                                                      \
                                                                       
            ds_sit_tot_turno_eleito qt_votos_nominais_validos_eleito   
Código IBGE                                                            
3550308                      ELEITO                          3169121   
3304557                      ELEITO                          1629319   
3106200                      ELEITO                           784307   
2927408                      ELEITO                           779408   
2304400                      ELEITO                           668652   
...                             ...                              ...   
3507209                      ELEITO                              501   
3555703                      ELEITO                              490   
4112603                      ELEITO                              470   
5101209                      ELEITO                              464   
5107883                      ELEITO                              409   

                                                                              \
                                                                               
            ds_genero_eleito ds_grau_instrucao_eleito ds_estado_civil_eleito   
Código IBGE                                                                    
3550308            MASCULINO        SUPERIOR COMPLETO          DIVORCIADO(A)   
3304557            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
3106200            MASCULINO      SUPERIOR INCOMPLETO              CASADO(A)   
2927408            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
2304400            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
...                      ...                      ...                    ...   
3507209            MASCULINO    ENSINO MÉDIO COMPLETO              CASADO(A)   
3555703             FEMININO    ENSINO MÉDIO COMPLETO            SOLTEIRO(A)   
4112603             FEMININO        SUPERIOR COMPLETO          DIVORCIADO(A)   
5101209            MASCULINO      SUPERIOR INCOMPLETO            SOLTEIRO(A)   
5107883        

In [74]:
despesas_orc_join = finbra_despesas_orc_pivot_soma_3.copy()
despesas_orc_join.index.name='Código IBGE'
despesas_orc_join.columns = pd.MultiIndex.from_tuples(
    [(x[0],'Despesas Orçamentárias | '+x[1],x[2]) for x in despesas_orc_join.columns]
)


despesas_por_func_join = finbra_despesas_por_func_pivot_soma.copy()
despesas_por_func_join.index.name='Código IBGE'
despesas_por_func_join.columns = pd.MultiIndex.from_tuples(
    [(x[0],'Despesas Por Função | '+x[1],x[2]) for x in despesas_por_func_join.columns]
)

In [75]:
despesas_orc_join = despesas_orc_join.merge(
    despesas_por_func_join
    ,how='inner'
    ,left_index=True
    ,right_index=True
)
despesas_orc_join = despesas_orc_join[sorted(despesas_orc_join.columns)]

In [81]:
df_base2 = df_base2.merge(
    despesas_orc_join
    ,how='left'
    ,left_index=True
    ,right_index=True
    #,suffixes=('')
)

In [82]:
df_base2.head()

\
                                                             
            nr_candidato_eleito        nm_candidato_eleito   
Código IBGE                                                  
1100015                      19                GIOVAN DAMO   
1100023                      51    CARLA GONÇALVES REZENDE   
1100031                      11         IZAEL DIAS MOREIRA   
1100049                      55  ADAILTON ANTUNES FERREIRA   
1100056                      43               LISETE MARTH   

                                                          \
                                                           
                             nm_ue sg_ue nr_turno_eleito   
Código IBGE                                                
1100015      ALTA FLORESTA D OESTE   310               1   
1100023                  ARIQUEMES    78               1   
1100031                     CABIXI   450               1   
1100049                     CACOAL    94               1   
1100056                 CEREJEIRAS   272               1   

                                                                      \
                                                                       
            ds_sit_tot_turno_eleito qt_votos_nominais_validos_eleito   
Código IBGE                                                            
1100015                      ELEITO                             7536   
1100023                      ELEITO                            13345   
1100031                      ELEITO                             2242   
1100049                      ELEITO                            25791   
1100056                      ELEITO                             3741   

                                                                              \
                                                                               
            ds_genero_eleito ds_grau_instrucao_eleito ds_estado_civil_eleito   
Código IBGE                                                                    
1100015                  NaN                      NaN                    NaN   
1100023             FEMININO        SUPERIOR COMPLETO              CASADO(A)   
1100031            MASCULINO        SUPERIOR COMPLETO            SOLTEIRO(A)   
1100049            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
1100056             FEMININO        SUPERIOR COMPLETO          DIVORCIADO(A)   

                                                                 \
                                                                  
            ds_cor_raca_eleito               ds_ocupacao_eleito   
Código IBGE                                                       
1100015                    NaN                              NaN   
1100023                 BRANCA                       ENFERMEIRO   
1100031                 BRANCA  PROFESSOR DE ENSINO FUNDAMENTAL   
1100049                  PARDA                         DEPUTADO   
1100056                 BRANCA                         PREFEITO   

                                                                            \
                                                                             
            total_votos_municipio_eleito  %_eleito nr_candidato_nao_eleito   
Código IBGE                                                                  
1100015                            13120  0.574390                    12.0   
1100023                            40383  0.330461                    77.0   
1100031                             3630  0.617631                    45.0   
1100049                            42251  0.610423                    25.0   
1100056                             8926  0.419113                    25.0   

                                                                        \
                                                                         
                           nm_candidato_nao_eleito nr_turno_nao_eleito   
Código IBGE                                                  

In [84]:
df_base2.to_excel(proj_root+'/Dados/Interim/base_final_v1.xlsx',engine='openpyxl')

# Filtragem de colunas para apenas as desejadas

In [85]:
df_base2.head()

\
                                                             
            nr_candidato_eleito        nm_candidato_eleito   
Código IBGE                                                  
1100015                      19                GIOVAN DAMO   
1100023                      51    CARLA GONÇALVES REZENDE   
1100031                      11         IZAEL DIAS MOREIRA   
1100049                      55  ADAILTON ANTUNES FERREIRA   
1100056                      43               LISETE MARTH   

                                                          \
                                                           
                             nm_ue sg_ue nr_turno_eleito   
Código IBGE                                                
1100015      ALTA FLORESTA D OESTE   310               1   
1100023                  ARIQUEMES    78               1   
1100031                     CABIXI   450               1   
1100049                     CACOAL    94               1   
1100056                 CEREJEIRAS   272               1   

                                                                      \
                                                                       
            ds_sit_tot_turno_eleito qt_votos_nominais_validos_eleito   
Código IBGE                                                            
1100015                      ELEITO                             7536   
1100023                      ELEITO                            13345   
1100031                      ELEITO                             2242   
1100049                      ELEITO                            25791   
1100056                      ELEITO                             3741   

                                                                              \
                                                                               
            ds_genero_eleito ds_grau_instrucao_eleito ds_estado_civil_eleito   
Código IBGE                                                                    
1100015                  NaN                      NaN                    NaN   
1100023             FEMININO        SUPERIOR COMPLETO              CASADO(A)   
1100031            MASCULINO        SUPERIOR COMPLETO            SOLTEIRO(A)   
1100049            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
1100056             FEMININO        SUPERIOR COMPLETO          DIVORCIADO(A)   

                                                                 \
                                                                  
            ds_cor_raca_eleito               ds_ocupacao_eleito   
Código IBGE                                                       
1100015                    NaN                              NaN   
1100023                 BRANCA                       ENFERMEIRO   
1100031                 BRANCA  PROFESSOR DE ENSINO FUNDAMENTAL   
1100049                  PARDA                         DEPUTADO   
1100056                 BRANCA                         PREFEITO   

                                                                            \
                                                                             
            total_votos_municipio_eleito  %_eleito nr_candidato_nao_eleito   
Código IBGE                                                                  
1100015                            13120  0.574390                    12.0   
1100023                            40383  0.330461                    77.0   
1100031                             3630  0.617631                    45.0   
1100049                            42251  0.610423                    25.0   
1100056                             8926  0.419113                    25.0   

                                                                        \
                                                                         
                           nm_candidato_nao_eleito nr_turno_nao_eleito   
Código IBGE                                                  

In [86]:
for c in df_base2.columns:
    print(c)

('', '', 'nr_candidato_eleito')
('', '', 'nm_candidato_eleito')
('', '', 'nm_ue')
('', '', 'sg_ue')
('', '', 'nr_turno_eleito')
('', '', 'ds_sit_tot_turno_eleito')
('', '', 'qt_votos_nominais_validos_eleito')
('', '', 'ds_genero_eleito')
('', '', 'ds_grau_instrucao_eleito')
('', '', 'ds_estado_civil_eleito')
('', '', 'ds_cor_raca_eleito')
('', '', 'ds_ocupacao_eleito')
('', '', 'total_votos_municipio_eleito')
('', '', '%_eleito')
('', '', 'nr_candidato_nao_eleito')
('', '', 'nm_candidato_nao_eleito')
('', '', 'nr_turno_nao_eleito')
('', '', 'ds_sit_tot_turno_nao_eleito')
('', '', 'qt_votos_nominais_validos_nao_eleito')
('', '', 'ds_genero_nao_eleito')
('', '', 'ds_grau_instrucao_nao_eleito')
('', '', 'ds_estado_civil_nao_eleito')
('', '', 'ds_cor_raca_nao_eleito')
('', '', 'ds_ocupacao_nao_eleito')
('', '', 'total_votos_municipio_nao_eleito')
('', '', '%_nao_eleito')
('', '', 'diferenca_pp')
('', '', 'flag_dif_5pp')
('', '', 'flag_dif_10pp')
('', '', 'flag_genero_diferente_eleito_nao_e

In [87]:
colunas_desejadas = []
for c in df_base2.columns:
    if (c[0]=='') | (c[1].split('|')[-1].strip()==c[2]) | (c[2]=='transferências a instituições privadas sem fins lucrativos'):
        colunas_desejadas.append(c)

colunas_desejadas += [
    ('Despesas Empenhadas', 'Despesas Por Função | 12 - Educação','12.365 - Educação Infantil')
    ,('Despesas Empenhadas', 'Despesas Por Função | 12 - Educação','12.361 - Ensino Fundamental')
    ,('Despesas Empenhadas', 'Despesas Por Função | 12 - Educação','12.362 - Ensino Médio')
    ,('Despesas Empenhadas', 'Despesas Por Função | 12 - Educação','12.364 - Ensino Superior')
    ,('Despesas Empenhadas', 'Despesas Por Função | 12 - Educação','12.367 - Educação Especial')
    ,('Despesas Pagas', 'Despesas Por Função | 12 - Educação','12.365 - Educação Infantil')
    ,('Despesas Pagas', 'Despesas Por Função | 12 - Educação','12.361 - Ensino Fundamental')
    ,('Despesas Pagas', 'Despesas Por Função | 12 - Educação','12.362 - Ensino Médio')
    ,('Despesas Pagas', 'Despesas Por Função | 12 - Educação','12.364 - Ensino Superior')
    ,('Despesas Pagas', 'Despesas Por Função | 12 - Educação','12.367 - Educação Especial')
]

In [95]:
colunas_desejadas[:34]

[('', '', 'nr_candidato_eleito'),
 ('', '', 'nm_candidato_eleito'),
 ('', '', 'nm_ue'),
 ('', '', 'sg_ue'),
 ('', '', 'nr_turno_eleito'),
 ('', '', 'ds_sit_tot_turno_eleito'),
 ('', '', 'qt_votos_nominais_validos_eleito'),
 ('', '', 'ds_genero_eleito'),
 ('', '', 'ds_grau_instrucao_eleito'),
 ('', '', 'ds_estado_civil_eleito'),
 ('', '', 'ds_cor_raca_eleito'),
 ('', '', 'ds_ocupacao_eleito'),
 ('', '', 'total_votos_municipio_eleito'),
 ('', '', '%_eleito'),
 ('', '', 'nr_candidato_nao_eleito'),
 ('', '', 'nm_candidato_nao_eleito'),
 ('', '', 'nr_turno_nao_eleito'),
 ('', '', 'ds_sit_tot_turno_nao_eleito'),
 ('', '', 'qt_votos_nominais_validos_nao_eleito'),
 ('', '', 'ds_genero_nao_eleito'),
 ('', '', 'ds_grau_instrucao_nao_eleito'),
 ('', '', 'ds_estado_civil_nao_eleito'),
 ('', '', 'ds_cor_raca_nao_eleito'),
 ('', '', 'ds_ocupacao_nao_eleito'),
 ('', '', 'total_votos_municipio_nao_eleito'),
 ('', '', '%_nao_eleito'),
 ('', '', 'diferenca_pp'),
 ('', '', 'flag_dif_5pp'),
 ('', '', 'fla

In [96]:
df_base3 = df_base2[colunas_desejadas[:33]+sorted(colunas_desejadas[33:])].copy()

In [97]:
df_base3.head()

\
                                                             
            nr_candidato_eleito        nm_candidato_eleito   
Código IBGE                                                  
1100015                      19                GIOVAN DAMO   
1100023                      51    CARLA GONÇALVES REZENDE   
1100031                      11         IZAEL DIAS MOREIRA   
1100049                      55  ADAILTON ANTUNES FERREIRA   
1100056                      43               LISETE MARTH   

                                                          \
                                                           
                             nm_ue sg_ue nr_turno_eleito   
Código IBGE                                                
1100015      ALTA FLORESTA D OESTE   310               1   
1100023                  ARIQUEMES    78               1   
1100031                     CABIXI   450               1   
1100049                     CACOAL    94               1   
1100056                 CEREJEIRAS   272               1   

                                                                      \
                                                                       
            ds_sit_tot_turno_eleito qt_votos_nominais_validos_eleito   
Código IBGE                                                            
1100015                      ELEITO                             7536   
1100023                      ELEITO                            13345   
1100031                      ELEITO                             2242   
1100049                      ELEITO                            25791   
1100056                      ELEITO                             3741   

                                                                              \
                                                                               
            ds_genero_eleito ds_grau_instrucao_eleito ds_estado_civil_eleito   
Código IBGE                                                                    
1100015                  NaN                      NaN                    NaN   
1100023             FEMININO        SUPERIOR COMPLETO              CASADO(A)   
1100031            MASCULINO        SUPERIOR COMPLETO            SOLTEIRO(A)   
1100049            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
1100056             FEMININO        SUPERIOR COMPLETO          DIVORCIADO(A)   

                                                                 \
                                                                  
            ds_cor_raca_eleito               ds_ocupacao_eleito   
Código IBGE                                                       
1100015                    NaN                              NaN   
1100023                 BRANCA                       ENFERMEIRO   
1100031                 BRANCA  PROFESSOR DE ENSINO FUNDAMENTAL   
1100049                  PARDA                         DEPUTADO   
1100056                 BRANCA                         PREFEITO   

                                                                            \
                                                                             
            total_votos_municipio_eleito  %_eleito nr_candidato_nao_eleito   
Código IBGE                                                                  
1100015                            13120  0.574390                    12.0   
1100023                            40383  0.330461                    77.0   
1100031                             3630  0.617631                    45.0   
1100049                            42251  0.610423                    25.0   
1100056                             8926  0.419113                    25.0   

                                                                        \
                                                                         
                           nm_candidato_nao_eleito nr_turno_nao_eleito   
Código IBGE                                                  

In [98]:
df_base3.to_excel(proj_root+'/Dados/Interim/base_final_v2.xlsx',engine='openpyxl')

In [99]:
df_base_share = df_base3.copy()

In [100]:
for c in df_base3.columns:
    if (c[0]=='Despesas Empenhadas') & \
    (c[1].split('|')[0].strip()=='Despesas Orçamentárias') & \
    (c[2]!='Total Geral da Despesa'):
        df_base_share[c] = df_base_share[c]/df_base_share[
            ('Despesas Empenhadas', 'Despesas Orçamentárias | Total Geral da Despesa', 'Total Geral da Despesa')
        ]

for c in df_base3.columns:
    if (c[0]=='Despesas Pagas') & \
    (c[1].split('|')[0].strip()=='Despesas Orçamentárias') & \
    (c[2]!='Total Geral da Despesa'):
        df_base_share[c] = df_base_share[c]/df_base_share[
            ('Despesas Pagas', 'Despesas Orçamentárias | Total Geral da Despesa', 'Total Geral da Despesa')
        ]
        
for c in df_base3.columns:
    if (c[0]=='Despesas Empenhadas') & \
    (c[1].split('|')[0].strip()=='Despesas Por Função') & \
    (c[2]!='Despesas Exceto Intraorçamentárias'):
        df_base_share[c] = df_base_share[c]/df_base_share[
            ('Despesas Empenhadas', 'Despesas Por Função | Despesas Exceto Intraorçamentárias', 'Despesas Exceto Intraorçamentárias')
        ]

for c in df_base3.columns:
    if (c[0]=='Despesas Pagas') & \
    (c[1].split('|')[0].strip()=='Despesas Por Função') & \
    (c[2]!='Despesas Exceto Intraorçamentárias'):
        df_base_share[c] = df_base_share[c]/df_base_share[
            ('Despesas Pagas', 'Despesas Por Função | Despesas Exceto Intraorçamentárias', 'Despesas Exceto Intraorçamentárias')
        ]

In [101]:
df_base_share.head()

\
                                                             
            nr_candidato_eleito        nm_candidato_eleito   
Código IBGE                                                  
1100015                      19                GIOVAN DAMO   
1100023                      51    CARLA GONÇALVES REZENDE   
1100031                      11         IZAEL DIAS MOREIRA   
1100049                      55  ADAILTON ANTUNES FERREIRA   
1100056                      43               LISETE MARTH   

                                                          \
                                                           
                             nm_ue sg_ue nr_turno_eleito   
Código IBGE                                                
1100015      ALTA FLORESTA D OESTE   310               1   
1100023                  ARIQUEMES    78               1   
1100031                     CABIXI   450               1   
1100049                     CACOAL    94               1   
1100056                 CEREJEIRAS   272               1   

                                                                      \
                                                                       
            ds_sit_tot_turno_eleito qt_votos_nominais_validos_eleito   
Código IBGE                                                            
1100015                      ELEITO                             7536   
1100023                      ELEITO                            13345   
1100031                      ELEITO                             2242   
1100049                      ELEITO                            25791   
1100056                      ELEITO                             3741   

                                                                              \
                                                                               
            ds_genero_eleito ds_grau_instrucao_eleito ds_estado_civil_eleito   
Código IBGE                                                                    
1100015                  NaN                      NaN                    NaN   
1100023             FEMININO        SUPERIOR COMPLETO              CASADO(A)   
1100031            MASCULINO        SUPERIOR COMPLETO            SOLTEIRO(A)   
1100049            MASCULINO        SUPERIOR COMPLETO              CASADO(A)   
1100056             FEMININO        SUPERIOR COMPLETO          DIVORCIADO(A)   

                                                                 \
                                                                  
            ds_cor_raca_eleito               ds_ocupacao_eleito   
Código IBGE                                                       
1100015                    NaN                              NaN   
1100023                 BRANCA                       ENFERMEIRO   
1100031                 BRANCA  PROFESSOR DE ENSINO FUNDAMENTAL   
1100049                  PARDA                         DEPUTADO   
1100056                 BRANCA                         PREFEITO   

                                                                            \
                                                                             
            total_votos_municipio_eleito  %_eleito nr_candidato_nao_eleito   
Código IBGE                                                                  
1100015                            13120  0.574390                    12.0   
1100023                            40383  0.330461                    77.0   
1100031                             3630  0.617631                    45.0   
1100049                            42251  0.610423                    25.0   
1100056                             8926  0.419113                    25.0   

                                                                        \
                                                                         
                           nm_candidato_nao_eleito nr_turno_nao_eleito   
Código IBGE                                                  

In [103]:
df_base_share.to_excel(proj_root+'/Dados/Processed/base_final_v2_share.xlsx',engine='openpyxl')